In [34]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [36]:
df = pd.read_csv("sample_text.csv")
df.shape

(36, 2)

In [37]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel
8,10 simple exercises to stay fit at home,Health
9,Organic foods that boost your immune system,Health


### Step 1 : Create source embeddings for the text column

In [38]:
from sentence_transformers import SentenceTransformer

In [39]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [40]:
vectors.shape

(36, 768)

In [41]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [42]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [43]:
index.add(vectors)

In [44]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000196F34FB750> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [46]:
search_query = "I want to buy home made medicine"
#search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
new_vec = encoder.encode(search_query)
new_vec.shape

(768,)

In [47]:
import numpy as np
svec = np.array(new_vec).reshape(1,-1)
svec.shape

(1, 768)

In [49]:
faiss.normalize_L2(svec)
#http://localhost:8888/notebooks/faiss_tutorial.ipynb#Step-5:-Search-for-similar-vector-in-the-FAISS-index-created

### Step 5: Search for similar vector in the FAISS index created

In [50]:
distances, I = index.search(svec, k=2)
distances

array([[1.2806205, 1.4241502]], dtype=float32)

In [51]:
I

array([[16,  9]], dtype=int64)

In [52]:
I.tolist()

[[16, 9]]

In [53]:
row_indices = I.tolist()[0]
row_indices

[16, 9]

In [54]:
df.loc[row_indices]

,text,category
16,Home remedies for common colds and flu,Health
9,Organic foods that boost your immune system,Health


In [55]:
search_query

'I want to buy home made medicine'

You can see that the two results from the dataframe are similar to a search_query